# 延迟初始化
:label:`sec_lazy_init`

到目前为止，我们似乎在设置网络时有些随意。具体来说，我们做了以下一些看似不合理的事情，但它们却能正常工作：

* 我们定义了网络架构，而没有指定输入维度。
* 我们添加了层，而没有指定前一层的输出维度。
* 甚至在提供足够的信息以确定模型应包含多少参数之前，我们就“初始化”了这些参数。

你可能会惊讶于我们的代码居然能够运行。毕竟，深度学习框架无法知道网络的输入维度是多少。这里的诀窍是框架*推迟初始化*，等到第一次将数据通过模型传递时，动态推断每一层的大小。

稍后，在处理卷积神经网络时，这种技术会变得更加方便，因为输入维度（例如图像的分辨率）会影响后续每一层的维度。因此，能够在编写代码时不需要知道维度的具体值就能设置参数，可以大大简化指定和随后修改模型的任务。接下来，我们将深入探讨初始化的机制。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

首先，让我们实例化一个MLP。

In [2]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

此时，网络不可能知道输入层权重的维度，因为输入维度仍然未知。

因此框架尚未初始化任何参数。
我们通过尝试访问以下参数来确认。

In [3]:
net[0].weight

<UninitializedParameter>

接下来让我们将数据通过网络传递，以便框架最终初始化参数。

In [4]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])

一旦我们知道输入的维度，
20，
框架就可以通过插入20的值来确定第一层权重矩阵的形状。
在识别了第一层的形状后，框架继续
到第二层，
依此类推，直到计算图中的所有形状都已知。
请注意，在这种情况下，
只有第一层需要延迟初始化，
但框架是按顺序初始化的。
一旦所有参数的形状都已知，
框架最终可以初始化这些参数。

以下方法
通过网络
传递虚拟输入
进行一次预运行
以推断所有参数的形状
并随后初始化这些参数。
当不希望使用默认的随机初始化时，后续会用到此方法。

In [5]:
@d2l.add_to_class(d2l.Module)  #@save
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)

## 摘要

惰性初始化很方便，可以让框架自动推断参数形状，便于修改架构，并消除一个常见的错误来源。
我们可以通过模型传递数据，使框架最终初始化参数。


## 练习

1. 如果你只指定了第一层的输入维度而没有指定后续层的输入维度会发生什么？你会得到立即初始化吗？
1. 如果你指定了不匹配的维度会发生什么？
1. 如果你的输入维度是可变的，你需要做什么？提示：看看参数绑定。

[讨论](https://discuss.d2l.ai/t/8092)